## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Kaitangata,NZ,-46.2817,169.8464,54.00,67,5,3.11,clear sky,2022-05-16 00:27:35
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,74.34,83,77,21.43,broken clouds,2022-05-16 00:27:35
2,2,Vangaindrano,MG,-23.3500,47.6000,65.93,89,32,2.15,light rain,2022-05-16 00:27:36
3,3,Barrow,US,71.2906,-156.7887,33.82,80,75,9.22,broken clouds,2022-05-16 00:27:36
4,4,Novokayakent,RU,42.3937,47.9861,62.78,58,46,6.31,scattered clouds,2022-05-16 00:27:37


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
11,11,Hithadhoo,MV,-0.6000,73.0833,82.69,76,99,12.15,overcast clouds,2022-05-16 00:27:39
12,12,Arlit,NE,18.7369,7.3853,82.15,15,0,4.43,clear sky,2022-05-16 00:27:40
19,19,Georgetown,MY,5.4112,100.3354,80.53,92,20,5.75,few clouds,2022-05-16 00:27:42
21,21,Palu,ID,-0.8917,119.8707,82.24,82,26,1.03,scattered clouds,2022-05-16 00:27:43
29,29,Salalah,OM,17.0151,54.0924,82.49,47,2,6.91,clear sky,2022-05-16 00:27:45
30,30,West Bay,KY,19.3667,-81.4167,81.07,78,40,8.05,scattered clouds,2022-05-16 00:27:46
31,31,Kapaa,US,22.0752,-159.3190,82.38,73,0,9.22,clear sky,2022-05-16 00:27:46
41,41,Butaritari,KI,3.0707,172.7902,81.64,75,12,10.45,few clouds,2022-05-16 00:27:50
43,43,Vila Velha,BR,-20.3297,-40.2925,75.15,73,40,8.05,scattered clouds,2022-05-16 00:27:50
45,45,Rikitea,PF,-23.1203,-134.9692,76.91,70,43,18.90,scattered clouds,2022-05-16 00:27:52


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
Date                   214
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
Date                   214
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Hithadhoo,MV,82.69,overcast clouds,-0.6000,73.0833,
12,Arlit,NE,82.15,clear sky,18.7369,7.3853,
19,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
21,Palu,ID,82.24,scattered clouds,-0.8917,119.8707,
29,Salalah,OM,82.49,clear sky,17.0151,54.0924,
30,West Bay,KY,81.07,scattered clouds,19.3667,-81.4167,
31,Kapaa,US,82.38,clear sky,22.0752,-159.3190,
41,Butaritari,KI,81.64,few clouds,3.0707,172.7902,
43,Vila Velha,BR,75.15,scattered clouds,-20.3297,-40.2925,
45,Rikitea,PF,76.91,scattered clouds,-23.1203,-134.9692,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")
        

Scoop Guest House
Case de Passage De la Prefecture
Cititel Penang
Hotel Santika Palu
HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel
Calypso Cove
Sheraton Kauai Coconut Beach Resort
Isles Sunset Lodge
Hotel Vitória Palace
People ThankYou
Galaxy Lake Resort
Maringe Lagoon Lodge
Grage Hotel Bengkulu
Best Western Lawrenceburg Inn
Hilo Hawaiian Hotel
Maui Seaside Hotel
Courtyard by Marriott Wichita at Old Town
Hotel not found... skipping.
Mediterrânea Hotel e Restaurante
El rranchito michuacan de ocampos
Paradise Inn
Villa Enata
Kiang Dao Resort
Divyansh Srivastava
Copamarina Beach Resort & Spa
Hôtel Le Lagon
Khách Sạn Mường Thanh Luxury Cà Mau
Bungalows Guesthouse
Takalvan Motel
JABIR HOTEL
Hotel not found... skipping.
Hotel Beatitude de Grand Lahou
Queens Guest Inn
Holiday Inn Express Greenville, an IHG Hotel
HOTEL IMAN
Hampton Inn Ukiah
Sapriadi Appy
Lords Inn Somnath
Mondkor Town, Bagbo Chiefdom, Bo District
Hotel Wixel
Hotel not found... skipping.
LIKWALE LODGE
Six Acres Bed & Breakfast
Nusa

In [14]:
import numpy as np

# Replace empty strings in Hotel Name column with NaN
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Hithadhoo,MV,82.69,overcast clouds,-0.6000,73.0833,Scoop Guest House
12,Arlit,NE,82.15,clear sky,18.7369,7.3853,Case de Passage De la Prefecture
19,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
21,Palu,ID,82.24,scattered clouds,-0.8917,119.8707,Hotel Santika Palu
29,Salalah,OM,82.49,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))